In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from PIL import Image
import cv2
from fastai.vision import ImageList
from fastai.vision import *
from fastai.tabular import *
from skimage.transform import resize

import re

import time
import gc
# from tqdm import tqdm
from scipy.ndimage import median_filter

import os
# os.listdir('/kaggle')

In [2]:
%%time
test_data = pd.read_parquet('./test_image_data_0.parquet', engine='fastparquet')
test_data = pd.concat([test_data, 
                           pd.read_parquet('./test_image_data_1.parquet', 
                                           engine='fastparquet')], ignore_index=True)
test_data = pd.concat([test_data, 
                           pd.read_parquet('./test_image_data_2.parquet', 
                                           engine='fastparquet')], ignore_index=True)
test_data = pd.concat([test_data, 
                           pd.read_parquet('./test_image_data_3.parquet', 
                                           engine='fastparquet')], ignore_index=True)
gc.collect()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Columns: 32333 entries, image_id to 32331
dtypes: object(1), uint8(32332)
memory usage: 379.1+ KB
Wall time: 2min 1s


In [3]:
test_data['label'] = 1
test_data['index'] = test_data.index
test_data.head()

,image_id,0,1,2,3,4,5,6,7,8,...,32324,32325,32326,32327,32328,32329,32330,32331,label,index
0,Test_0,247,253,253,252,252,252,252,253,253,...,254,254,254,254,253,253,252,250,1,0
1,Test_1,253,253,253,253,253,253,253,253,253,...,255,255,255,255,255,255,255,255,1,1
2,Test_2,253,253,253,253,253,252,251,252,252,...,255,255,255,255,255,254,253,252,1,2
3,Test_3,199,227,236,241,238,242,245,243,243,...,226,224,222,227,220,208,194,136,1,3
4,Test_4,243,246,251,252,251,249,250,250,251,...,252,252,252,253,253,253,253,253,1,4


In [5]:
'''img_name = image[0]
    image = np.reshape(image[1:], (137, 236))
    image = (255 - image).astype(np.float32) / 255.
    image = median_filter(image, size=5)
    
    assert image.ndim == 2
    is_black = image > threshold

    is_black_vertical = np.sum(is_black, axis=0) > 0
    is_black_horizontal = np.sum(is_black, axis=1) > 0
    left = np.argmax(is_black_horizontal)
    right = np.argmax(is_black_horizontal[::-1])
    top = np.argmax(is_black_vertical)
    bottom = np.argmax(is_black_vertical[::-1])
    height, width = image.shape
    cropped_image = image[0 :height - right + pad, 
                          0:width - bottom + pad]
    cropped_image = Image.fromarray(cropped_image*255).convert('RGB').resize((size,size))
    cropped_image.save('/kaggle/working/test_images/'+img_name+'.jpg')'''

"img_name = image[0]\n    image = np.reshape(image[1:], (137, 236))\n    image = (255 - image).astype(np.float32) / 255.\n    image = median_filter(image, size=5)\n    \n    assert image.ndim == 2\n    is_black = image > threshold\n\n    is_black_vertical = np.sum(is_black, axis=0) > 0\n    is_black_horizontal = np.sum(is_black, axis=1) > 0\n    left = np.argmax(is_black_horizontal)\n    right = np.argmax(is_black_horizontal[::-1])\n    top = np.argmax(is_black_vertical)\n    bottom = np.argmax(is_black_vertical[::-1])\n    height, width = image.shape\n    cropped_image = image[0 :height - right + pad, \n                          0:width - bottom + pad]\n    cropped_image = Image.fromarray(cropped_image*255).convert('RGB').resize((size,size))\n    cropped_image.save('/kaggle/working/test_images/'+img_name+'.jpg')"

In [4]:
threshold=20./255.
pad=10
size = 128

class PixelList(ImageList):
    def open(self,index):
        # from PIL import Image
        
        regex = re.compile(r'\d+')
        fn = re.findall(regex,index)
        df_fn = self.inner_df[self.inner_df.index.values == int(fn[0])]
        image = df_fn.drop(labels=['image_id','index','label'],axis=1).values
        image = image.reshape(137, 236)
        image = (255 - image).astype (np.float32) / 255.
        # image = median_filter(image, size=5)
        assert image.ndim == 2
        is_black = image > threshold

        is_black_vertical = np.sum(is_black, axis=0) > 0
        is_black_horizontal = np.sum(is_black, axis=1) > 0
        left = np.argmax(is_black_horizontal)
        right = np.argmax(is_black_horizontal[::-1])
        top = np.argmax(is_black_vertical)
        bottom = np.argmax(is_black_vertical[::-1])
        height, width = image.shape
        cropped_image = image[0 :height - right + pad, 0:width - bottom + pad]
        
        
        cropped_image = cropped_image*255
        cropped_image = resize(cropped_image, (size, size))
        # cropped_image = cropped_image.resize((size, size))
        cropped_image = np.stack((cropped_image,)*3,axis=-1)
        
        # cropped_image = Image.fromarray(cropped_image*255).convert('RGB').resize((size,size))
        
        
        # image = np.stack((cropped_image,)*3,axis=-1)
        # from fastai import vision
        return vision.Image(pil2tensor(cropped_image,np.float32).div_(255))

In [5]:
tfms = get_transforms(max_zoom=1.3, 
                      max_rotate=25, 
                      max_warp=0.2, 
                      p_affine=0.2)

In [6]:
test = PixelList.from_df(test_data,'.',cols='index')
src = (PixelList.from_df(test_data,'.',cols='index')
       .split_none()
       .label_from_df(cols='label')
       .add_test(test
                 # , label=0
                ))
src

LabelLists;

Train: LabelList (12 items)
x: PixelList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: CategoryList
1,1,1,1,1
Path: .;

Valid: LabelList (0 items)
x: PixelList

y: CategoryList

Path: .;

Test: LabelList (12 items)
x: PixelList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: EmptyLabelList
,,,,
Path: .

In [10]:
data = (src.databunch(bs=6)
        # .transform(tfms,size=128)
        .normalize()
       )

BrokenPipeError: [Errno 32] Broken pipe

In [8]:
data.show_batch()

NameError: name 'data' is not defined

In [11]:
from fastai import *
from fastai.vision import *
learn = load_learner('./','fastai_resnet_fl_wtd_filter')
learn.data = data

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.loss.BCEWithLogitsLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute

In [12]:
preds = learn.get_preds(ds_type=DatasetType.Test)

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.6/site-packages/fastai/data_block.py", line 651, in __getitem__
    if self.item is None: x,y = self.x[idxs],self.y[idxs]
  File "/opt/conda/lib/python3.6/site-packages/fastai/data_block.py", line 120, in __getitem__
    if isinstance(idxs, Integral): return self.get(idxs)
  File "/opt/conda/lib/python3.6/site-packages/fastai/vision/data.py", line 271, in get
    res = self.open(fn)
  F

In [13]:
%%time
predictions = []
for i in range(test_data.shape[0]):
    row = test_data.iloc[i].values
    predictions.append(get_predictions(row))
    gc.collect()

NameError: name 'get_predictions' is not defined

In [14]:
submission = pd.DataFrame(predictions, columns = ['row_id',
                                                  '_grapheme_root',
                                                  '_vowel_diacritic', 
                                                  '_consonant_diacritic'])
submission = submission.melt(id_vars=['row_id']).sort_values(['row_id'])
submission['row_id'] = submission['row_id']+submission['variable']
submission = submission[['row_id','value']]
submission.columns = ['row_id','target']
submission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 2 columns):
row_id    0 non-null object
target    0 non-null object
dtypes: object(2)
memory usage: 0.0+ bytes


In [15]:
submission.to_csv('submission.csv', index=False)

In [16]:
# submission